## Collect experimental data and process csv

In [ ]:
from pathlib import Path

from natsort import natsorted
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from neuroprocessing.scripts.parse_csv import parse_csv, process_data


In [ ]:
def _get_sync_info(sync_csv_path):
    """Get dict of sync info (stim time, etc) from the sync csv file
    """
    df_daq = parse_csv(sync_csv_path)

    df_frames = process_data(
        df_daq,
        col_camera="camera",
        col_stim="button"
    )

    stim_onset_frame = int(df_frames.loc[df_frames["stimulated"], "frame"].iloc[0])
    df_frames.set_index("frame", inplace=True)
    # print info
    print(f"Stimulus onset frame: {stim_onset_frame}")
    print(f"Stimulus onset time (s): {df_frames.loc[stim_onset_frame, 'time']}")
    stim_duration_frames = sum(df_frames['stimulated'])
    frame_time_s = df_frames.loc[stim_onset_frame, 'frametime']
    framerate_hz = 1/frame_time_s

    sync_info = {
        "stim_onset_frame": stim_onset_frame,
        "stim_duration_frames": stim_duration_frames,
        "frame_time_s": frame_time_s,
        "framerate_hz": framerate_hz
    }
    print(f"Stimulus duration (frames): {stim_duration_frames}")
    print(f"Stimulus duration (s): {stim_duration_frames/framerate_hz}")
    print(f"Frame duration (ms): {frame_time_s*1000}")
    print(f"Framerate (Hz): %.2f" % (framerate_hz))
    return sync_info

def process_trial(tif_paths, decimation_factor,sync_csv):
    """Process single trial
    
    Process a single imaging trial and save the processed tiff stack to the same directory as the original tiff stack.

    An imaging trial can be split into 2+ videos due to tiff file size limits.

    Stimulus is assumed to be in the first video.

    Inputs:
        tif_paths: str or list of str
            Path to the tiff stack(s) to process. You may have multiple stacks due to Tif size limits. If a list of paths is provided, the stacks will be concatenated along the time axis.
            Assume that the stimulation happens in the first stack that is loaded
        decimation_factor: int
            Factor by which the stack has been decimated
            **TODO** can probably hardcode the decimation factor below
        sync_csv: Path or int
            If Path, path to the sync csv file. If int, the frame number of the stimulus onset.
    """

    # process sync file
    if type(sync_csv) == int:
        print(f"WARNING: no CSV sync file provided, using {sync_csv} as stimulus onset")
        sync_info = {
            "stim_onset_frame": sync_csv,
            "stim_duration_frames": 100,
            "frame_time_s": .1,
            "framerate_hz": 10
        }
    else:
        sync_info = _get_sync_info(sync_csv)

    if type(tif_paths) == str:
        tif_paths = [tif_paths]

    # iterate through tifs and concatenate stacks
    stack_stim = []
    first_movie = True
    for tif_path in tif_paths:
        stack = io.imread(tif_path)
        CROP_PX = 20
        BOTTOM_PERCENTILE = 5
        # crop image a little bit to remove the black border
        stack = stack[:, CROP_PX:-CROP_PX, CROP_PX:-CROP_PX]

        # not processing baseline frames for now
        if first_movie:
            # if first movie, add only the frames after the stimulus onset
            stack_stim.append(stack[int((sync_info["stim_onset_frame"]+100)/decimation_factor):, :, :])
            first_movie = False
        else:
            # if subsequent movie, add all frames
            stack_stim.append(stack)

    stack_stim = np.concatenate(stack_stim, axis=0)
    # find bottom X% of pixels in image
    bottom10 = np.percentile(stack_stim, BOTTOM_PERCENTILE, axis=(1,2), keepdims=True).astype(np.uint16)
    # subtract bottom X% from all pixels
    stack_stim -= bottom10
    stack_stim -= stack_stim.min(axis=0, keepdims=True)
    stack_stim[stack_stim > 30000] = 0

    # save tiff stack with the name of the first tiff in the stack
    io.imsave(Path(tif_path).parent / ("processed_" + Path(tif_paths[0]).name), stack_stim)


# Run
* This processes a single trial

In [ ]:

# location where all experimental data is kept
dir_experiments = Path("/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections")

# path to a particular (or set of) experiment(s) for analysis
date = "2024-03-06"
expt = 'Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1' # 'RHL_hist' # "RHL_hist" # LHL_saline
fp_csv = natsorted((dir_experiments/date/expt).glob("*.csv"))[0]
fp_tifs = natsorted((dir_experiments/date/expt).glob("aligned*.tif"))

decimation_factor = 8
img_d = process_trial(fp_tifs, decimation_factor, sync_csv=fp_csv)
